In [ ]:
%%capture
!pip install rdkit

In [ ]:
#bunch of cool stuff
import pandas as pd
import numpy as np
import os
import rdkit
import io
from PIL import Image
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import AllChem

In [ ]:
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 800,800

In [ ]:
!git clone https://github.com/AstraZeneca/peptide-tools.git # get the peptide-tools repository to use the SMARTS .dat file
substructures_smarts_file = 'peptide-tools/pIChemiSt/pichemist/data/smarts/legacy/smarts_pKaMatcher.dat' # set the path to the SMARTS file

In [ ]:
def ionizable_groups(SMILES):
  matches = {} # where the ion centers will be stored. I tried to save the indices to highlight them but some of the positions defined in the pKaMatcher file are wrong. To-do!
  compound_mol = Chem.MolFromSmiles(SMILES)
  global ion_centers
  ion_centers = []

  with open(substructures_smarts_file, 'r') as f:
    for line in f.readlines(): # read the file line by line
      ln = line.strip().split()
      if not ln or line.startswith('#'): continue # skip empty lines and comments

      substr_mol = Chem.MolFromSmarts(ln[1]) # get the substructure in the current line as Mol
      substr_matches = compound_mol.GetSubstructMatches(substr_mol) # tuple of matches with the inputed molecule

      if len(substr_matches) != 0: # if the matches tuple has at least one item
        print(ln[1])
        for substr in range(len(substr_matches)): # for every matched substructure
          print(f"Ciclo {substr}")
          print(substr_matches[substr])
          compound_center = substr_matches[substr][int(ln[2])-1] # get the atom index for the ionization center
          print(int(ln[2]))
          if compound_center in ion_centers: # if the center has already been added, skip it
            continue
          else: # if it hasn't been added
            ion_centers.append(compound_center) # add it for subsequent checks
            matches[compound_center] = ln[5] # add the center and its type to the matches dictionary
            if len(ln) > 6: # this checks if the substructure has more than one ionization center (e.g. catechol-like compounds)
              compound_center = substr_matches[substr][int(ln[6])-1] # get the atom index for the second ionization center
              ion_centers.append(compound_center) # add it for subsequent checks
              matches[compound_center] = ln[9] # add the center and its type to the matches dictionary

  total_acid = 0
  total_base = 0
  for key, value in matches.items():
    if value == 'acid':
      total_acid += 1
    elif value == 'base':
      total_base += 1


  if total_acid + total_base > 0:
    ions = "Ionizable"
  else:
    ions = "Non-ionizable"

  return ions

In [ ]:
#test = 'CC(C)CC(C(=O)NC(CC(=O)O)C(=O)NC(CC(C)C)C(=O)O)NC(=O)C(C(C)C)NC(=O)C(CCCCN)NC(=O)C(CC(=O)N)NC(=O)C(CCSC)NC(=O)C(CC(=O)O)NC(=O)C(CC1=CN=CN1)N ' # this is an example to try different SMILES
test = 'CN1CCC23C4C1CC5=C2C(=C(C=C5)O)OC3C(C=C4)O '
ionizable_groups(test)

In [ ]:
%%capture
df = pd.read_csv('NAPRORE_v4.2.csv')

df['IONIZABLE'] = df['SMILES'].apply(lambda x: ionizable_groups(x))

In [ ]:
df

In [ ]:
df.to_csv('NAPRORE_v4.2.csv')